<a href="https://colab.research.google.com/github/sundaybest3/nounsmart_spelling/blob/main/spelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gradio
!pip install pandas
!pip install random

In [ ]:
import gradio as gr
import pandas as pd
import random

# Load the CSV file
csv_url = "https://raw.githubusercontent.com/sundaybest3/NounSmart_Streamlit/main/nouns_CE_visang.csv"
df = pd.read_csv(csv_url)

# Inspect the columns
expected_columns = ["Page", "Word", "Countability", "Plural"]
missing_columns = [col for col in expected_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"The following required columns are missing in the CSV: {', '.join(missing_columns)}")

# Filter only countable nouns
df_countable = df[df["Countability"].str.lower() == "countable"].copy()

# Initialize game state
game_state = {
    "nickname": None,
    "page": None,
    "current_noun": None,
    "remaining_nouns": None,
    "score": 0,
    "trials": 0,
    "total_nouns": 0
}

# Function to reset the game state
def reset_game_state():
    game_state["current_noun"] = None
    game_state["remaining_nouns"] = None
    game_state["score"] = 0
    game_state["trials"] = 0
    game_state["total_nouns"] = 0

# Function to filter countable nouns by page and prepare the list of nouns
def initialize_nouns(page):
    reset_game_state()
    filtered_df = df_countable[df_countable["Page"] == page]
    if filtered_df.empty:
        return "No countable nouns available for this page. Please select a different page."
    game_state["remaining_nouns"] = filtered_df.sample(frac=1).to_dict(orient="records")
    game_state["total_nouns"] = len(filtered_df)
    return show_random_noun()

# Function to show a random noun
def show_random_noun():
    if not game_state["remaining_nouns"]:
        return f"🎉 Great job, {game_state['nickname']}! All nouns in page {game_state['page']} have been answered correctly. (Score: {game_state['score']}/{game_state['total_nouns']})"

    game_state["current_noun"] = game_state["remaining_nouns"].pop()
    return game_state["current_noun"]["Word"]

# Function to check user's plural input
def check_plural(user_input):
    if not game_state.get("current_noun"):
        return "Please click 'Show the Noun' first."

    correct_plural = game_state["current_noun"]["Plural"].strip().lower()
    game_state["trials"] += 1

    if user_input.lower().strip() == correct_plural:
        game_state["score"] += 1
        feedback = f"✅ Correct! The plural of '{game_state['current_noun']['Word']}' is '{correct_plural}'. (Score: {game_state['score']}/{game_state['trials']}) Click 'Show the Noun' to continue."
    else:
        # Add the noun back to the remaining list for retry
        game_state["remaining_nouns"].insert(0, game_state["current_noun"])
        feedback = f"❌ Incorrect. The plural of '{game_state['current_noun']['Word']}' is '{correct_plural}'. (Score: {game_state['score']}/{game_state['trials']}) It will appear again. Click 'Show the Noun' to continue."

    if not game_state["remaining_nouns"]:
        feedback += f" 🎉 Great job, {game_state['nickname']}! All nouns in page {game_state['page']} have been answered correctly. (Score: {game_state['score']}/{game_state['total_nouns']})"

    return feedback

# Gradio interface
def main_interface(nickname, page, user_input=None):
    if page != game_state.get("page"):
        game_state["nickname"] = nickname
        game_state["page"] = page
        return initialize_nouns(page)

    if user_input is None:
        return show_random_noun()

    feedback = check_plural(user_input)
    return feedback

# Prepare page summary for dropdown
def get_page_summary():
    summary = df_countable.groupby("Page").size().reset_index(name="Total")
    summary["Display"] = summary["Page"].astype(str) + " (" + summary["Total"].astype(str) + " nouns)"
    return summary["Page"], summary["Display"]

pages, page_display = get_page_summary()
page_choices = dict(zip(page_display, pages))

# Gradio app components
with gr.Blocks() as app:
    nickname = gr.Textbox(label="Enter your nickname:")
    page = gr.Dropdown(choices=list(page_choices.keys()), label="Select your page:")
    show_button = gr.Button("Show the Noun")
    noun_display = gr.Textbox(label="Noun:", interactive=False)
    plural_input = gr.Textbox(label="Enter the plural form:")
    submit_button = gr.Button("Check the Answer")
    feedback_display = gr.Textbox(label="Feedback and Score:", interactive=False)

    # Map display back to page for processing
    def map_page_choice(display_choice):
        return page_choices[display_choice]

    show_button.click(fn=lambda n, p: main_interface(n, map_page_choice(p)), inputs=[nickname, page], outputs=[noun_display])
    submit_button.click(fn=lambda n, p, c: main_interface(n, map_page_choice(p), c), inputs=[nickname, page, plural_input], outputs=[feedback_display])

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://10403eb0a85698ff42.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
